In [26]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
%autoreload now

In [28]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-08137aa2-e69b-5e74-8390-7997329b1336"
os.environ["WANDB_API_KEY"] = ""

# Download and convert data

In [29]:
from document_segmentation.pagexml.annotations.renate_analysis import (
    RenateAnalysis,
    RenateAnalysisInv,
)
from document_segmentation.pagexml.annotations.sheet import Sheet
from document_segmentation.settings import RENATE_ANALYSIS_SHEETS

sheets: list[Sheet] = [RenateAnalysis()] + [
    RenateAnalysisInv(sheet) for sheet in RENATE_ANALYSIS_SHEETS
]

In [30]:
from itertools import islice

from tqdm import tqdm

from document_segmentation.pagexml.datamodel.inventory import Inventory

N = None

inventories: dict[Sheet, list[Inventory]] = {
    sheet: list(
        tqdm(
            islice(sheet.inventories(), N),
            desc="Loading inventories",
            unit="inventory",
            total=N or len(list(sheet.inventory_numbers())),
        )
    )
    for sheet in sheets
}
inventories

Loading inventories:  75%|███████▌  | 54/72 [00:48<00:08,  2.03inventory/s]ERROR:root:Failed to load inventory 8171: 404 Client Error: Not Found for url: https://hucdrive.huc.knaw.nl/HTR/obp-v2-pagexml-leon-metadata-trimmed-2023-11/81711.zip
ERROR:root:Failed to load inventory 8260: 404 Client Error: Not Found for url: https://hucdrive.huc.knaw.nl/HTR/obp-v2-pagexml-leon-metadata-trimmed-2023-11/82601.zip
Loading inventories: 100%|██████████| 1/1 [00:00<00:00, 38479.85inventory/s]


{<document_segmentation.pagexml.annotations.renate_analysis.RenateAnalysis at 0x469e4d590>: [Inventory(inv_nr=1060, inventory_part=, pages=478 pages),
  Inventory(inv_nr=1066, inventory_part=, pages=994 pages),
  Inventory(inv_nr=1070, inventory_part=, pages=1322 pages),
  Inventory(inv_nr=1072, inventory_part=, pages=942 pages),
  Inventory(inv_nr=1074, inventory_part=, pages=724 pages),
  Inventory(inv_nr=1081, inventory_part=, pages=816 pages),
  Inventory(inv_nr=1083, inventory_part=, pages=1176 pages),
  Inventory(inv_nr=1088, inventory_part=, pages=1232 pages),
  Inventory(inv_nr=1105, inventory_part=, pages=1092 pages),
  Inventory(inv_nr=1108, inventory_part=, pages=1284 pages),
  Inventory(inv_nr=1118, inventory_part=, pages=1280 pages),
  Inventory(inv_nr=1139, inventory_part=, pages=1896 pages),
  Inventory(inv_nr=1141, inventory_part=, pages=1324 pages),
  Inventory(inv_nr=1147, inventory_part=, pages=1566 pages),
  Inventory(inv_nr=1150, inventory_part=, pages=1086 pages),

In [31]:
for sheet, sheet_inventories in inventories.items():
    for inventory in sheet_inventories:
        sheet.annotate_inventory(inventory)
inventories

{<document_segmentation.pagexml.annotations.renate_analysis.RenateAnalysis at 0x469e4d590>: [Inventory(inv_nr=1060, inventory_part=, pages=478 pages),
  Inventory(inv_nr=1066, inventory_part=, pages=994 pages),
  Inventory(inv_nr=1070, inventory_part=, pages=1322 pages),
  Inventory(inv_nr=1072, inventory_part=, pages=942 pages),
  Inventory(inv_nr=1074, inventory_part=, pages=724 pages),
  Inventory(inv_nr=1081, inventory_part=, pages=816 pages),
  Inventory(inv_nr=1083, inventory_part=, pages=1176 pages),
  Inventory(inv_nr=1088, inventory_part=, pages=1232 pages),
  Inventory(inv_nr=1105, inventory_part=, pages=1092 pages),
  Inventory(inv_nr=1108, inventory_part=, pages=1284 pages),
  Inventory(inv_nr=1118, inventory_part=, pages=1280 pages),
  Inventory(inv_nr=1139, inventory_part=, pages=1896 pages),
  Inventory(inv_nr=1141, inventory_part=, pages=1324 pages),
  Inventory(inv_nr=1147, inventory_part=, pages=1566 pages),
  Inventory(inv_nr=1150, inventory_part=, pages=1086 pages),

# Load Data

In [32]:
TRAINING_DATA = 0.8

In [33]:
inventories

{<document_segmentation.pagexml.annotations.renate_analysis.RenateAnalysis at 0x469e4d590>: [Inventory(inv_nr=1060, inventory_part=, pages=478 pages),
  Inventory(inv_nr=1066, inventory_part=, pages=994 pages),
  Inventory(inv_nr=1070, inventory_part=, pages=1322 pages),
  Inventory(inv_nr=1072, inventory_part=, pages=942 pages),
  Inventory(inv_nr=1074, inventory_part=, pages=724 pages),
  Inventory(inv_nr=1081, inventory_part=, pages=816 pages),
  Inventory(inv_nr=1083, inventory_part=, pages=1176 pages),
  Inventory(inv_nr=1088, inventory_part=, pages=1232 pages),
  Inventory(inv_nr=1105, inventory_part=, pages=1092 pages),
  Inventory(inv_nr=1108, inventory_part=, pages=1284 pages),
  Inventory(inv_nr=1118, inventory_part=, pages=1280 pages),
  Inventory(inv_nr=1139, inventory_part=, pages=1896 pages),
  Inventory(inv_nr=1141, inventory_part=, pages=1324 pages),
  Inventory(inv_nr=1147, inventory_part=, pages=1566 pages),
  Inventory(inv_nr=1150, inventory_part=, pages=1086 pages),

In [34]:
from collections import defaultdict

labelled_inventories: dict[Sheet, list[Inventory]] = defaultdict(list)

for sheet, sheet_inventories in inventories.items():
    for inventory in sheet_inventories:
        for labelled in inventory.labelled_inventories():
            labelled_inventories[sheet].append(labelled.preprocess())
labelled_inventories

defaultdict(list,
            {<document_segmentation.pagexml.annotations.renate_analysis.RenateAnalysis at 0x469e4d590>: [Inventory(inv_nr=1060, inventory_part=, pages=14 pages),
              Inventory(inv_nr=1066, inventory_part=, pages=1 pages),
              Inventory(inv_nr=1066, inventory_part=, pages=2 pages),
              Inventory(inv_nr=1070, inventory_part=, pages=3 pages),
              Inventory(inv_nr=1072, inventory_part=, pages=7 pages),
              Inventory(inv_nr=1072, inventory_part=, pages=12 pages),
              Inventory(inv_nr=1072, inventory_part=, pages=3 pages),
              Inventory(inv_nr=1074, inventory_part=, pages=16 pages),
              Inventory(inv_nr=1081, inventory_part=, pages=35 pages),
              Inventory(inv_nr=1083, inventory_part=, pages=36 pages),
              Inventory(inv_nr=1088, inventory_part=, pages=60 pages),
              Inventory(inv_nr=1105, inventory_part=, pages=3 pages),
              Inventory(inv_nr=1108, inventor

In [35]:
import random

for sheet, sheet_inventories in labelled_inventories.items():
    random.shuffle(sheet_inventories)
labelled_inventories

defaultdict(list,
            {<document_segmentation.pagexml.annotations.renate_analysis.RenateAnalysis at 0x469e4d590>: [Inventory(inv_nr=1210, inventory_part=, pages=15 pages),
              Inventory(inv_nr=3952, inventory_part=, pages=8 pages),
              Inventory(inv_nr=1141, inventory_part=, pages=4 pages),
              Inventory(inv_nr=8696, inventory_part=, pages=27 pages),
              Inventory(inv_nr=8820, inventory_part=, pages=9 pages),
              Inventory(inv_nr=1268, inventory_part=, pages=15 pages),
              Inventory(inv_nr=1509, inventory_part=, pages=30 pages),
              Inventory(inv_nr=1072, inventory_part=, pages=12 pages),
              Inventory(inv_nr=8935, inventory_part=, pages=79 pages),
              Inventory(inv_nr=1188, inventory_part=, pages=42 pages),
              Inventory(inv_nr=3095, inventory_part=, pages=23 pages),
              Inventory(inv_nr=1118, inventory_part=, pages=12 pages),
              Inventory(inv_nr=1072, inven

In [36]:
# the first 80% of the RenateAnalysis sheet + the first of the RenateAnalysisInv sheets
_renate_analysis_inventories = list(labelled_inventories.values())[0]
split = int(TRAINING_DATA * len(_renate_analysis_inventories))
_renate_analysis_per_inv_1 = list(labelled_inventories.values())[1]

training_data = _renate_analysis_inventories[:split] + _renate_analysis_per_inv_1
training_data

[Inventory(inv_nr=1210, inventory_part=, pages=15 pages),
 Inventory(inv_nr=3952, inventory_part=, pages=8 pages),
 Inventory(inv_nr=1141, inventory_part=, pages=4 pages),
 Inventory(inv_nr=8696, inventory_part=, pages=27 pages),
 Inventory(inv_nr=8820, inventory_part=, pages=9 pages),
 Inventory(inv_nr=1268, inventory_part=, pages=15 pages),
 Inventory(inv_nr=1509, inventory_part=, pages=30 pages),
 Inventory(inv_nr=1072, inventory_part=, pages=12 pages),
 Inventory(inv_nr=8935, inventory_part=, pages=79 pages),
 Inventory(inv_nr=1188, inventory_part=, pages=42 pages),
 Inventory(inv_nr=3095, inventory_part=, pages=23 pages),
 Inventory(inv_nr=1118, inventory_part=, pages=12 pages),
 Inventory(inv_nr=1072, inventory_part=, pages=7 pages),
 Inventory(inv_nr=1060, inventory_part=, pages=14 pages),
 Inventory(inv_nr=8526, inventory_part=, pages=14 pages),
 Inventory(inv_nr=1105, inventory_part=, pages=3 pages),
 Inventory(inv_nr=1083, inventory_part=, pages=36 pages),
 Inventory(inv_nr=1

In [37]:
validation_data = (
    _renate_analysis_inventories[split:] + list(labelled_inventories.values())[2]
)
validation_data

[Inventory(inv_nr=1239, inventory_part=, pages=2 pages),
 Inventory(inv_nr=3589, inventory_part=, pages=17 pages),
 Inventory(inv_nr=3553, inventory_part=, pages=68 pages),
 Inventory(inv_nr=1480, inventory_part=, pages=3 pages),
 Inventory(inv_nr=1139, inventory_part=, pages=4 pages),
 Inventory(inv_nr=1319, inventory_part=, pages=1 pages),
 Inventory(inv_nr=3524, inventory_part=, pages=86 pages),
 Inventory(inv_nr=1547, inventory_part=, pages=4 pages),
 Inventory(inv_nr=1635, inventory_part=, pages=3 pages),
 Inventory(inv_nr=8846, inventory_part=, pages=3 pages),
 Inventory(inv_nr=3174, inventory_part=, pages=4 pages),
 Inventory(inv_nr=1506, inventory_part=, pages=4 pages),
 Inventory(inv_nr=3248, inventory_part=, pages=26 pages),
 Inventory(inv_nr=1557, inventory_part=, pages=1662 pages)]

# Train Model

In [38]:
EPOCHS = 3

In [39]:
%autoreload now

In [40]:
from document_segmentation.model.page_sequence_tagger import PageSequenceTagger

tagger = PageSequenceTagger()

In [41]:
tagger._device

'mps'

In [42]:
tagger

PageSequenceTagger(
  (_page_embedding): PageEmbedding(
    (_region_model): RegionEmbeddingSentenceTransformer(
      (_transformer_model): SentenceTransformer(
        (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: RobertaModel 
        (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
      )
      (_region_type): Embedding(10, 16)
      (_linear): Linear(in_features=784, out_features=512, bias=True)
    )
    (_rnn): LSTM(512, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
    (_linear): Linear(in_features=512, out_features=256, bias=True)
  )
  (_rnn): LSTM(256, 256, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (_linear): Linear(in_features=512, out_featur

In [43]:
tagger.train_(training_data, validation_data, epochs=EPOCHS, shuffle=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Evaluating: 100%|██████████| 14/14 [01:13<00:00,  5.27s/batch]
        [4]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.
Evaluating: 100%|██████████| 14/14 [00:04<00:00,  2.83batch/s]
        [4]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.
Evaluating: 100%|██████████| 14/14 [00:04<00:00,  3.36batch/s]
        [4]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.
Evaluating: 100%|██████████| 14/14 [00:03<00:00,  4.14batch/s]
        [4]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.
Evaluating: 100%|██████████| 14/14 [00:03<00:00,  3.96batch/s]
        [4]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.
Evaluating: 100%|██████████| 14/14 

MulticlassAccuracy,▁█████████
epoch,▁▂▃▃▄▅▆▆▇█
inventory length,▁█▂▁▁▁▁▁▁▁▂▁█▁▁▁▁▁▁▁▁▂▁▁▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁
loss,▃█▂▂▁▂▂▂▂▂▂▂█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂
MulticlassAccuracy,0.8071
epoch,9
inventory length,2
loss,5.98615


# Evaluate Model

In [44]:
import sys

precision, recall, f1, accuracy = tagger.eval_(validation_data, sys.stdout)

Predicted	Actual	Page ID	Text	Scores


Evaluating:  14%|█▍        | 2/14 [00:00<00:03,  3.82batch/s]

BEGIN	BEGIN	NL-HaNA_1.04.02_1239_0447.jpg	N=a 1: Origineele brieff door d' h=ren Rijckloff v	[2.168033279303927e-05, 0.9998724460601807, 6.647569534834474e-05, 1.3269564988149796e-05, 2.6150573830818757e-05]
END	END	NL-HaNA_1.04.02_1239_0448.jpg	casje den schipper ter; met opschrift aen d'Ede:; 	[3.991579797002487e-05, 1.2969010640517808e-05, 9.660788055043668e-05, 0.999824583530426, 2.590228723420296e-05]
BEGIN	BEGIN	NL-HaNA_1.04.02_3589_0657.jpg	Extract uijt de Gehoudene; aantekening door den Op	[8.43961606733501e-06, 0.9998756647109985, 0.00010583172115730122, 1.3793595599054242e-06, 8.587797310610767e-06]
IN	IN	NL-HaNA_1.04.02_3589_0658.jpg	antwoord als eenvoudig, Ja of en een, mede; te bre	[5.6526965636294335e-05, 0.0005896739312447608, 0.9992331266403198, 6.726566789438948e-05, 5.341861105989665e-05]
IN	IN	NL-HaNA_1.04.02_3589_0659.jpg	heen te gaan, dog toen wierd hij nog; meer verleeg	[1.4688780538563151e-05, 8.870100282365456e-05, 0.9998283386230469, 5.295740265864879e-05, 1.52

Evaluating:  21%|██▏       | 3/14 [00:00<00:02,  3.97batch/s]

BEGIN	BEGIN	NL-HaNA_1.04.02_3553_0151.jpg	den 6e: Octebe ten te. cmbr . 1730	[9.162231435766444e-06, 0.9998743534088135, 0.00010523034870857373, 1.5826938124519074e-06, 9.692947060102597e-06]
IN	IN	NL-HaNA_1.04.02_3553_0152.jpg		[5.2160219638608396e-05, 0.0005202238680794835, 0.9993302822113037, 4.9538171879248694e-05, 4.782799442182295e-05]
IN	IN	NL-HaNA_1.04.02_3553_0153.jpg	gehouden door den Boek„; houder en Commissiant; de	[1.740316474752035e-05, 0.0001229253684869036, 0.9997826218605042, 5.914449138799682e-05, 1.7860473235487007e-05]
IN	IN	NL-HaNA_1.04.02_3553_0154.jpg	om na Landak te vertrekken, Ik schreef ook aan; sa	[1.1080003787355963e-05, 4.812778206542134e-05, 0.999885082244873, 4.387257285998203e-05, 1.1837028068839572e-05]
IN	IN	NL-HaNA_1.04.02_3553_0155.jpg	Waaren, ik otving gisteren en heeden tyding, dat g	[1.0454758921696339e-05, 4.272582373232581e-05, 0.9998923540115356, 4.3317941162968054e-05, 1.10607852548128e-05]
IN	IN	NL-HaNA_1.04.02_3553_0156.jpg	lieden beroepen z

Evaluating:  93%|█████████▎| 13/14 [00:01<00:00, 17.79batch/s]

BEGIN	BEGIN	NL-HaNA_1.04.02_3524_0903.jpg	Gehouden geduurende de Commissie; na Landak en Suc	[8.493379027640913e-06, 0.9998934268951416, 8.812387386569753e-05, 1.4076340448809788e-06, 8.481577424390707e-06]
IN	IN	NL-HaNA_1.04.02_3524_0904.jpg	was te voorschijn gekoomen om in iets order te ste	[5.537755714613013e-05, 0.0005273071001283824, 0.9992947578430176, 7.23847551853396e-05, 5.020510434405878e-05]
IN	IN	NL-HaNA_1.04.02_3524_0905.jpg	teegen den avond arreveerde de gezanten van Cherib	[1.5545921996817924e-05, 0.00010505718819331378, 0.9997982382774353, 6.310357275651768e-05, 1.8012100554187782e-05]
IN	IN	NL-HaNA_1.04.02_3524_0906.jpg	ook â Costij te arriveeren aan welken Letteren ons	[1.1002033716067672e-05, 4.6353878133231774e-05, 0.9998852014541626, 4.545428237179294e-05, 1.2042783964716364e-05]
IN	IN	NL-HaNA_1.04.02_3524_0907.jpg	en alle de daar omtrent leggende Plaatsen gesloote	[1.0336606464989018e-05, 4.245293530402705e-05, 0.9998927116394043, 4.349795199232176e-05, 1.10210721

Evaluating: 100%|██████████| 14/14 [00:04<00:00,  2.89batch/s]

IN	OUT	NL-HaNA_1.04.02_1557_1318.jpg		[2.0760404368047602e-05, 0.00012211030116304755, 0.9997197985649109, 0.00011460027599241585, 2.2729764168616384e-05]
IN	OUT	NL-HaNA_1.04.02_1557_1319.jpg		[2.075195334327873e-05, 0.00012245831021573395, 0.9997197985649109, 0.00011426633136579767, 2.2732667275704443e-05]
IN	OUT	NL-HaNA_1.04.02_1557_1320.jpg		[2.0746938389493152e-05, 0.00012317248911131173, 0.9997194409370422, 0.00011382091179257259, 2.274745020258706e-05]
IN	OUT	NL-HaNA_1.04.02_1557_1321.jpg		[2.0738454622915015e-05, 0.0001248748740181327, 0.9997186064720154, 0.00011302427446935326, 2.2779950086260214e-05]
IN	OUT	NL-HaNA_1.04.02_1557_1322.jpg		[2.0538665921776555e-05, 0.0001291411172132939, 0.9997180104255676, 0.00010960335202980787, 2.2686996089760214e-05]
IN	BEGIN	NL-HaNA_1.04.02_1557_1323.jpg	29:en maart p:r de fluijt; Bovenkarspel; Register 	[2.0053941625519656e-05, 0.00013900241174269468, 0.9997029900550842, 0.00011562047438928857, 2.2208447262528352e-05]
IN	IN	NL-HaNA_1.04.02_

In [45]:
import csv
import sys

from document_segmentation.pagexml.datamodel.label import Label

writer = csv.DictWriter(
    sys.stdout, fieldnames=["Metric"] + [label.name for label in Label], delimiter="\t"
)

writer.writeheader()

for metric in precision, recall, f1:
    assert metric.average is None

    scores: list[float] = metric.compute().tolist()
    writer.writerow(
        {"Metric": metric.__class__.__name__}
        | {label.name: f"{score:.4f}" for label, score in zip(Label, scores)}
    )

assert accuracy.average is not None

print(
    f"{accuracy.__class__.__name__} ({accuracy.average} average):\t{accuracy.compute().item():.4f}",
    file=sys.stdout,
)

        [4]]) classes have zero instances in both the predictions and the ground truth labels. Precision is still logged as zero.


Metric	UNK	BEGIN	IN	END	OUT
MulticlassPrecision	0.0000	0.9231	0.8059	0.8571	0.0000
MulticlassRecall	0.0000	0.1333	1.0000	0.0642	0.0000
MulticlassF1Score	0.0000	0.2330	0.8925	0.1194	0.0000
MulticlassAccuracy (micro average):	0.8071
